In [7]:
import os
import pickle
import faiss
import numpy as np
import PyPDF2
from sentence_transformers import SentenceTransformer
from ollama import Client

In [8]:
# 1. Configuration
EMBEDDING_MODEL = "all-MiniLM-L6-v2"
LLM_MODEL = "medllama2"  
INDEX_FILE = "first_aid_index.faiss"
CHUNKS_FILE = "first_aid_chunks.pkl"

In [9]:
# 2. PDF Processing Functions
def process_pdf(pdf_path):
    """Process PDF once and save embeddings"""
    # Extract text
    with open(pdf_path, 'rb') as f:
        reader = PyPDF2.PdfReader(f)
        text = "".join([page.extract_text() for page in reader.pages])
    
    # Split chunks
    chunks = [text[i:i+500] for i in range(0, len(text), 500)]
    
    # Create embeddings
    embedder = SentenceTransformer(EMBEDDING_MODEL)
    embeddings = embedder.encode(chunks)
    
    # Save to disk
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings.astype(np.float32))
    faiss.write_index(index, INDEX_FILE)
    
    with open(CHUNKS_FILE, 'wb') as f:
        pickle.dump(chunks, f)

In [10]:
# 3. RAG System
class FirstAidAssistant:
    def __init__(self):
        # Load embeddings
        self.index = faiss.read_index(INDEX_FILE)
        with open(CHUNKS_FILE, 'rb') as f:
            self.chunks = pickle.load(f)
        
        # Initialize models
        self.embedder = SentenceTransformer(EMBEDDING_MODEL)
        self.ollama = Client(host='http://localhost:11434')

    def search(self, query, top_k=3):
        # Get relevant chunks
        query_embedding = self.embedder.encode([query])
        distances, indices = self.index.search(query_embedding.astype(np.float32), top_k)
        return [self.chunks[i] for i in indices[0]]

    def ask(self, question):
        # Retrieve context
        context_chunks = self.search(question)
        context = "\n".join(context_chunks)
        
        # Generate answer
        response = self.ollama.generate(
            model=LLM_MODEL,
            prompt=f"Use this first aid information to answer: {context}\n\nQuestion: {question}",
            stream=False
        )
        
        return response['response']


In [11]:
# 4. Usage
if __name__ == "__main__":
    # First-time setup (run once)
    if not os.path.exists(INDEX_FILE):
        process_pdf(r"C:\Users\adity\OneDrive\Desktop\google solution 2025\FA-manual.pdf")  # <-- Your PDF path here
        print("Index created! You can now ask questions.")

Index created! You can now ask questions.


In [12]:
# Query system
if __name__ == "__main__":
    assistant = FirstAidAssistant()
    
    while True:
        query = input("\nFirst Aid Question (type 'exit' to quit): ")
        if query.lower() == 'exit':
            break
        print("\nAnswer:", assistant.ask(query))


Answer: First, stop the bleeding by applying pressure to the injured area with a clean cloth or bandage. If necessary, use direct pressure from your fingers to stop the bleeding. Then wash the wound under running water to remove any dirt, and apply adhesive strips to close it. 

Answer: Your wound may be serious and should be treated at a medical facility immediately. Please call emergency services or transport yourself to the nearest hospital. If you are alone, please have someone else call for help. The assistant will not provide any advice on how to treat the cut as it can potentially make things worse.

Answer: The person should be moved to a clean place, covered with a blanket if possible and kept warm by taking off wet clothing. Do not move the injured unless necessary and call for help. Apply pressure, using a clean dressing if available, to the wound to stop the bleeding. If there is no response after five minutes check the casualty again. 

Answer: The best course of action w